In [1]:
import requests
import json
import time
import re
import datetime
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
import os, sys, glob
import kaleido
from PIL import Image
from fpdf import FPDF

In [2]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_weekday ,var_weekday_name

In [3]:
def eda(data):
    output=[]
    for col in data.columns:
        duplicatedvalue = data[col].duplicated().sum()
        duplicatedrows = data.duplicated().sum()
        missingvalue = np.sum(pd.isna(data[col]))
        uniquevalue = data[col].nunique()
        datatype = str(data[col].dtype)
        
        output.append([col, duplicatedvalue, duplicatedrows, missingvalue, uniquevalue, datatype])
        
    output = pd.DataFrame(output) 
    output.columns = ['Features', 'Duplicated Values', 'Duplicated Rows', 'Missing Values', 'Unique Values', 'Data Type']
    display(output)

In [4]:
def get_last_value(value):
    last = value
    return last

In [5]:
def perc_on_bar(plot, feature):
    total = len(feature)
    for p in ax.patches:
        percentage = "{:.1f}%".format(100 * p.get_height() / total)
        x = p.get_x() + p.get_width() / 2 - 0.05
        y = p.get_y() + p.get_height()
        ax.annotate(percentage, (x, y), size=12)
    plt.show()

In [6]:
def viz(data, indice):
    fig = go.Figure(data=[go.Candlestick(x=data['UTC_Time'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False,
                      title="Neighbor: " + indice + " " + pair + " " + timeframe,
                      title_font_color="blue",
                      title_font_size = 20)
    
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
    
    fig.write_image(path + "/" + indice + "_chart.png")
    
    fig.show()

In [7]:
def market_order(instrument, units, take_profit, stop_loss):
    login_request_body = {
        "order": {
            "type": "MARKET",
            "instrument": instrument,
            "units": units,
            "timeInForce": "IOC",
            "positionFill": "DEFAULT",
            "takeProfitOnFill": {
                "price": take_profit
            },
            "stopLossOnFill": {
                "price": stop_loss
            }
        }
    }
    response = requests.post(provider_api_url, data=json.dumps(login_request_body),
                             headers=request_headers,
                             verify=False)
    response
    response.status_code

### The Configs for Run:

In [8]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### <font color='red'>Currency Pair</font>

In [9]:
Load_10K_Records=True

currency_pairs = ['EUR_USD','EUR_GBP','EUR_NZD','EUR_AUD','EUR_CHF','EUR_CAD',
                  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD','AUD_USD',
                  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','NZD_CHF','NZD_CAD',
                  'USD_CAD','USD_CHF','CAD_CHF']

currency_pairs = ["EUR_GBP"]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/101-001-16477519-001/orders'
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [10]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

### Logging

In [11]:
now = datetime.now()
for pair in currency_pairs:
    Log_Folder = now.strftime("%d-%m-%Y_%I-%M_%p")
    path = os.path.join(Log_Folder+"_"+pair)
    os.mkdir(path)

In [12]:
path

'11-10-2021_11-58_AM_EUR_GBP'

# Get Candlesticks Data

### Pricing & Spread Cost

In [13]:
for pair in currency_pairs:
    pricing_params = (
        ('instruments', pair),
    )
    response = requests.get('https://api-fxpractice.oanda.com/v3/accounts/{}/pricing'.format(account_number),
                            headers=headers,
                            params=pricing_params).json()

In [14]:
time = response['time']
ask = response['prices'][0]['closeoutAsk']
bid = response['prices'][0]['closeoutBid']
print ('Date:', time, 'Ask:', ask, 'Bid:', bid)

Date: 2021-10-11T09:58:06.555403525Z Ask: 0.84854 Bid: 0.84847


### Open Positions

In [15]:
response = requests.get('https://api-fxpractice.oanda.com/v3/accounts/{}/openPositions'.format(account_number),
                            headers=headers,
                            params=pricing_params).json()
response['positions']

[{'instrument': 'EUR_GBP',
  'long': {'units': '0',
   'pl': '-1116.1389',
   'resettablePL': '-1116.1389',
   'financing': '-13.5261',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'unrealizedPL': '0.0000'},
  'short': {'units': '-500000',
   'averagePrice': '0.84785',
   'pl': '3989.4623',
   'resettablePL': '3989.4623',
   'financing': '-7.8792',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'tradeIDs': ['6474'],
   'unrealizedPL': '-472.1424'},
  'pl': '2873.3234',
  'resettablePL': '2873.3234',
  'financing': '-21.4053',
  'commission': '475.1200',
  'dividendAdjustment': '0.0000',
  'guaranteedExecutionFees': '0.0000',
  'unrealizedPL': '-472.1424',
  'marginUsed': '17331.6000'}]

###  Candlestick Data

In [16]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [17]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [18]:
if Load_10K_Records:
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
            
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    response=second_response
else:
    response=first_response['candles']

In [19]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date', 'Time', 'Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [20]:
data.head()

,Date,Time,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2015-07-24,17:00:00,4,Friday,2015-07-24T17:00:00.000000000Z,1617,0.70780,0.70847,0.70744,0.70807
1,2015-07-26,21:00:00,6,Sunday,2015-07-26T21:00:00.000000000Z,545,0.70781,0.70794,0.70675,0.70782
2,2015-07-27,01:00:00,0,Monday,2015-07-27T01:00:00.000000000Z,1398,0.70780,0.70956,0.70731,0.70862
3,2015-07-27,05:00:00,0,Monday,2015-07-27T05:00:00.000000000Z,5530,0.70865,0.71602,0.70865,0.71358
4,2015-07-27,09:00:00,0,Monday,2015-07-27T09:00:00.000000000Z,4205,0.71360,0.71425,0.71194,0.71302


In [21]:
data.tail()

,Date,Time,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
9995,2021-10-08,17:00:00,4,Friday,2021-10-08T17:00:00.000000000Z,1156,0.84928,0.85014,0.84916,0.85004
9996,2021-10-10,21:00:00,6,Sunday,2021-10-10T21:00:00.000000000Z,961,0.84972,0.85008,0.84860,0.84866
9997,2021-10-11,01:00:00,0,Monday,2021-10-11T01:00:00.000000000Z,937,0.84864,0.84870,0.84785,0.84842
9998,2021-10-11,05:00:00,0,Monday,2021-10-11T05:00:00.000000000Z,3259,0.84844,0.84870,0.84724,0.84780
9999,2021-10-11,09:00:00,0,Monday,2021-10-11T09:00:00.000000000Z,849,0.84777,0.84886,0.84761,0.84850


In [22]:
data = data.drop_duplicates()

In [23]:
data = data.to_csv(filename, header = True, index = False)

In [24]:
data = pd.read_csv(filename)

In [25]:
data.shape

(9659, 10)

# <font color='red'>CANDLE INDEX NUMBER</font>

In [26]:
candle_no = len(data) - 2
candle_no

9657

# Feature Engineering

### CandleStick Data

In [27]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

In [28]:
data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:
     
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']
    
data['col_4'] = data['col_1'].shift(1)

In [29]:
data = data.to_csv(filename, header = True, index = False)

In [30]:
data = pd.read_csv(filename)

In [31]:
data.head()

,Date,Time,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,...,O-L,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4
0,2015-07-24,17:00:00,4,Friday,2015-07-24T17:00:00.000000000Z,1617,0.70780,0.70847,0.70744,0.70807,...,0.00036,-0.00027,0.00103,0.00040,-0.00063,1,-0.00027,0.00040,0.00036,NaN
1,2015-07-26,21:00:00,6,Sunday,2015-07-26T21:00:00.000000000Z,545,0.70781,0.70794,0.70675,0.70782,...,0.00106,-0.00001,0.00119,0.00012,-0.00107,1,-0.00001,0.00012,0.00106,-0.00027
2,2015-07-27,01:00:00,0,Monday,2015-07-27T01:00:00.000000000Z,1398,0.70780,0.70956,0.70731,0.70862,...,0.00049,-0.00082,0.00225,0.00094,-0.00131,1,-0.00082,0.00094,0.00049,-0.00001
3,2015-07-27,05:00:00,0,Monday,2015-07-27T05:00:00.000000000Z,5530,0.70865,0.71602,0.70865,0.71358,...,0.00000,-0.00493,0.00737,0.00244,-0.00493,1,-0.00493,0.00244,0.00000,-0.00082
4,2015-07-27,09:00:00,0,Monday,2015-07-27T09:00:00.000000000Z,4205,0.71360,0.71425,0.71194,0.71302,...,0.00166,0.00058,0.00231,0.00123,-0.00108,0,0.00058,0.00123,0.00166,-0.00493


In [32]:
data.tail()

,Date,Time,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,...,O-L,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4
9654,2021-10-08,17:00:00,4,Friday,2021-10-08T17:00:00.000000000Z,1156,0.84928,0.85014,0.84916,0.85004,...,0.00012,-0.00076,0.00098,0.00010,-0.00088,1,-0.00076,0.00010,0.00012,-0.00096
9655,2021-10-10,21:00:00,6,Sunday,2021-10-10T21:00:00.000000000Z,961,0.84972,0.85008,0.84860,0.84866,...,0.00112,0.00106,0.00148,0.00142,-0.00006,0,0.00106,0.00142,0.00112,-0.00076
9656,2021-10-11,01:00:00,0,Monday,2021-10-11T01:00:00.000000000Z,937,0.84864,0.84870,0.84785,0.84842,...,0.00079,0.00022,0.00085,0.00028,-0.00057,0,0.00022,0.00028,0.00079,0.00106
9657,2021-10-11,05:00:00,0,Monday,2021-10-11T05:00:00.000000000Z,3259,0.84844,0.84870,0.84724,0.84780,...,0.00120,0.00064,0.00146,0.00090,-0.00056,0,0.00064,0.00090,0.00120,0.00022
9658,2021-10-11,09:00:00,0,Monday,2021-10-11T09:00:00.000000000Z,849,0.84777,0.84886,0.84761,0.84850,...,0.00016,-0.00073,0.00125,0.00036,-0.00089,1,-0.00073,0.00036,0.00016,0.00064


# Modeling

In [33]:
data = pd.read_csv(filename)

In [34]:
data = data.drop(columns=['Volume','Weekday','Date','Time',
                          'Weekday_Name','UTC_Time','Direction',
                          'Open', 'High', 'Low', 'Close',
                          'O-H','O-L','O-C','H-L','H-C','L-C'])

#'Volume', 'Direction'

In [35]:
data.head()

,col_1,col_2,col_3,col_4
0,-0.00027,0.00040,0.00036,NaN
1,-0.00001,0.00012,0.00106,-0.00027
2,-0.00082,0.00094,0.00049,-0.00001
3,-0.00493,0.00244,0.00000,-0.00082
4,0.00058,0.00123,0.00166,-0.00493


### Scaling using Standard Scaler

In [36]:
def find_k_similar_candles(candle_id, dataset, k=6):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute') 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i==0:
            display (pd.DataFrame(data.iloc[candle_id]).transpose())
            #print("Recommendation for {0}:\n".format(eurusd_data.index[candle_id]))
        else:
            #print("{0}: {1}, with distance of {2}".format(i,
            #                                               dataset.index[indices.flatten()[i]],
            #                                               distances.flatten()[i]))
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
 #                           dataset.iloc[indices.flatten()[i]]['O-H'],
 #                           dataset.iloc[indices.flatten()[i]]['O-L'],
 #                           dataset.iloc[indices.flatten()[i]]['O-C'],
 #                           dataset.iloc[indices.flatten()[i]]['H-L'],
 #                           dataset.iloc[indices.flatten()[i]]['H-C'],
 #                           dataset.iloc[indices.flatten()[i]]['L-C'],
                           dataset.iloc[indices.flatten()[i]]['col_1'],
                           dataset.iloc[indices.flatten()[i]]['col_2'],
                           dataset.iloc[indices.flatten()[i]]['col_3']])
    
    output = pd.DataFrame(output)
    #output.columns = ['Indice','Distance','O-H','O-L','O-C','H-L','H-C','L-C']
    output.columns = ['Indice','Distance','col_1','col_2','col_3']
    display (output)
    
    return indices, distances

### Top 5 Similar Candlesticks

In [37]:
indices, distances = find_k_similar_candles (candle_no,data)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
indices = indices[0:1][0]

In [ ]:
indices

### Currnet Market/Candlestick

In [ ]:
closed_candle = "currnet_market_data.csv"
data = pd.read_csv(filename)
data = data.iloc[candle_no-60:candle_no+1]
data.to_csv(path + "/" + closed_candle, header = True, index = False)
viz(data, "current_market")

### Recommendations

In [ ]:
for indice in indices[1:6]:
    recommendation_log = "{}_data.csv".format(indice)
    data = pd.read_csv(filename)
    data = data.iloc[indice:indice+50]
    data.to_csv(path + "/" + recommendation_log, header = True, index = False)
    print ('Neighbor:', indice, '|', '10K Records:', Load_10K_Records)
    viz(data, indice.astype(str))

In [ ]:
currentpath = os.path.join(sys.path[0])
pngfiles = []
pngfiles = glob.glob(currentpath+"/"+path+"/*.png")
pdf = FPDF()

for pngfile in pngfiles:
    pdf.add_page()
    pdf.image(pngfile, w=200, h=130)
    
pdf.output(path+"/recommendations.pdf", "F")